# Assignment - Week 2 Perceptron/Adaline

Jettapol Tuetrakul

jt5326@nyu.edu

## Instruction

(Note: Use the Scikit-learn library only if you need to scale the data, implement Perceptron and Adaline using gradient descent procedure without using sklearn.)
The programming assignment for the Perceptron/Adaline mode is

Download the sample Python code that comes with the assigned textbook, Python Machine Learning*.

Use the Iris dataset that's referenced in the text. (add page reference)

You can use the sample code from the text as a starting point or you can write your own code from scratch.

Pick two classes of data (i.e., two species of Iris) and two features from the four in the dataset, so that the data for two species are linearly separable using the features that you have chosen.

Then Apply the Perceptron and Adaline models to the classes/feature that you have chosen and report your results.

5. Repeat Step 4) using three features at a time.

6. Repeat Step 4) using all four features at the same time.

7. Pick two classes of data (i.e., two species of Iris) and two features from the four in the dataset, so that the data for two species are NOT linearly separable using the features that you have chosen.

Then Apply the Perceptron and Adaline models to the classes/features that you have chosen and report your results.

8. Repeat Step 7) using three features at a time.

9. Repeat Step 7) using all four feature at the same time.

10. Compare your results for the Perceptron and Adaline models.

11. Submit your results as an ipython notebook (i.e. in ipynb format).

You can use the example code in file M1_Python ML Basics.ipynb uploaded to the modules 1 & 2.

*Python Machine Learning: Machine Learning and Deep Learning with Python, scikit-learn,

## Download the datasets and prepare environments

Get Data Zip File

In [1]:
url = "https://archive.ics.uci.edu/static/public/53/iris.zip"

In [2]:
import requests, zipfile
from io import BytesIO
request = requests.get(url)
file = zipfile.ZipFile(BytesIO(request.content))
file.extractall()

Import Necessary labraries

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
df = pd.read_csv("iris.data",header=None,encoding="utf-8")
df.head()

In [ ]:
df.rename(columns={0: 'sepal_length', 1: 'sepal_width', 2: 'petal_length', 3: 'petal_width', 4: 'class_label'}, inplace=True)

In [ ]:
df.head()

## 4.) Pick two classes of data (i.e., two species of Iris) and two features from the four in the dataset, so that the data for two species are linearly separable using the features that you have chosen.

Selected classes (2): 
- Iris-setosa
- Iris-versicolor

Selected features (2): 
- Sepal length
- Petal length

Extract the first 100 class labels that correspond to the 50 Iris-setosa and 50 Iris-versicolor flowers

In [ ]:
y = df.loc[:99,"class_label"].values
y

Convert the class labels into the two integer class labels

In [ ]:
y = np.where(y=="Iris-setosa",-1,1)
y

In [ ]:
y.size

Extract sepal length and petal length

In [ ]:
X = df.loc[0:99, ["sepal_length","petal_length"]].values

In [ ]:
X.shape

Plot the data

In [ ]:
plt.scatter(X[:50,0], X[:50,1], color="red", marker="o", label="setosa")
plt.scatter(X[50:100,0], X[50:100,1], color="blue", marker="x", label="versicolor")
plt.xlabel("sepal length (cm)")
plt.ylabel("petal length (cm)")
plt.legend(loc="upper left")
plt.title("Setosa vs. Versicolor (sepal length/petal length)")
plt.show()

The graph shows that the two data are clearly linearly seperable.

### Perceptron Implementation

In [ ]:
# Code excerpts from Python Machine Learning 3rd Edition
# Implementing Perceptron
class Perceptron(object):
    """Perceptron classifier.

    Parameters
    ------------
    eta : float
      Learning rate (between 0.0 and 1.0)
    n_iter : int
      Passes over the training dataset.
    random_state : int
      Random number generator seed for random weight
      initialization.

    Attributes
    -----------
    w_ : 1d-array
      Weights after fitting.
    errors_ : list
      Number of misclassifications (updates) in each epoch.

    """
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_examples, n_features]
          Training vectors, where n_examples is the number of examples and
          n_features is the number of features.
        y : array-like, shape = [n_examples]
          Target values.

        Returns
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.errors_ = []

        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self

    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, -1)
        
    def evaluate(self, X, y):
        """Evaluate the model"""
        predictions = self.predict(X).reshape(-1)
        accuracy = np.sum(predictions == y) / y.shape[0]
        return accuracy

In [ ]:
ppn = Perceptron(eta=0.1, n_iter=10)
ppn.fit(X,y)
plt.plot(range(1, len(ppn.errors_)+1), ppn.errors_, marker="o")
plt.xlabel("epochs")
plt.ylabel("# of updates")
plt.title("Misclassification errors on each number of epochs")
plt.show()

## Evaluating the model

In [ ]:
train_acc = ppn.evaluate(X,y)
print("training set accuracy: %.2f%%" % (train_acc*100))

As shown in the plot, perceptron converged after the sixth epoch and can classify data with 100% accuracy.


In [ ]:
from matplotlib.colors import ListedColormap
def plot_decision_regions(X, y, classifier, resolution=0.02):
    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution), 
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    # plot class examples
    for idx, cl in enumerate(np.unique(y)):
         plt.scatter(x=X[y == cl, 0], 
         y=X[y == cl, 1], 
                     alpha=0.8, 
                     c=colors[idx], 
                     marker=markers[idx], 
                     label=cl, 
                     edgecolor='black')

In [ ]:
plot_decision_regions(X, y, classifier=ppn)
plt.xlabel("sepal length (cm)")
plt.ylabel("petal length (cm)")
plt.legend(["setosa","versicolor"],loc="upper left")
plt.title("Perceptron: setosa vs. versicolor (2 features)")
plt.show()

As we can see in the plot, the perceptron learned a decision boundary that is able to 
classify all flower examples in the Iris training subset perfectly.


### Adaline Implementation

In [ ]:
# Code excerpts from Python Machine Learning - 3rd Edition
# Adaline with Stochastic Gradient Descent

class AdalineSGD(object):
    """ADAptive LInear NEuron classifier.
    Parameters
    ------------
    eta : float
        Learning rate (between 0.0 and 1.0)
    n_iter : int
        Passes over the training dataset.
    shuffle : bool (default: True)
        Shuffles training data every epoch if True to prevent  
        cycles.
    random_state : int
        Random number generator seed for random weight  
        initialization.
    Attributes
    -----------
    w_ : 1d-array
        Weights after fitting.
    cost_ : list
        Sum-of-squares cost function value averaged over all
        training examples in each epoch.
    """
    def __init__(self, eta=0.01, n_iter=10, 
              shuffle=True, random_state=None):
        self.eta = eta
        self.n_iter = n_iter
        self.w_initialized = False
        self.shuffle = shuffle
        self.random_state = random_state
    def fit(self, X, y):
        """ Fit training data.
        Parameters
        ----------
        X : {array-like}, shape = [n_examples, n_features]
            Training vectors, where n_examples is the number of  
            examples and n_features is the number of features.
        y : array-like, shape = [n_examples]
            Target values.
        Returns
        -------
        self : object
        """
        self._initialize_weights(X.shape[1])
        self.cost_ = []
        for i in range(self.n_iter):
            if self.shuffle:
                X, y = self._shuffle(X, y)
            cost = []
            for xi, target in zip(X, y):
                cost.append(self._update_weights(xi, target))
            avg_cost = sum(cost) / len(y)
            self.cost_.append(avg_cost)
        return self
    def partial_fit(self, X, y):
        """Fit training data without reinitializing the weights"""
        if not self.w_initialized:
            self._initialize_weights(X.shape[1])
        if y.ravel().shape[0] > 1:
            for xi, target in zip(X, y):
                self._update_weights(xi, target)
        else:
            self._update_weights(X, y)
        return self
    def _shuffle(self, X, y):
        """Shuffle training data"""
        r = self.rgen.permutation(len(y))
        return X[r], y[r]
    def _initialize_weights(self, m):
        """Initialize weights to small random numbers"""
        self.rgen = np.random.RandomState(self.random_state)
        self.w_ = self.rgen.normal(loc=0.0, scale=0.01,
                                   size=1 + m)
        self.w_initialized = True
    def _update_weights(self, xi, target):
        """Apply Adaline learning rule to update the weights"""
        output = self.activation(self.net_input(xi))
        error = (target - output)
        self.w_[1:] += self.eta * xi.dot(error)
        self.w_[0] += self.eta * error
        cost = 0.5 * error**2
        return cost
    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]
    def activation(self, X):
        """Compute linear activation"""
        return X
    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.activation(self.net_input(X))
                        >= 0.0, 1, -1)
    def evaluate(self, X, y):
        """Evaluate the model"""
        predictions = self.predict(X).reshape(-1)
        accuracy = np.sum(predictions == y) / y.shape[0]
        return accuracy

First we need to standardize the data

In [ ]:
X_std = np.copy(X)
X_std[:,0] = (X[:,0] - X[:,0].mean()) / X[:,0].std()
X_std[:,1] = (X[:,1] - X[:,1].mean()) / X[:,1].std()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

ada1 = AdalineSGD(n_iter=10, eta=0.01).fit(X_std, y)
ax[0].plot(range(1, len(ada1.cost_) + 1), np.log10(ada1.cost_), marker='o')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('log(Sum-squared-error)')
ax[0].set_title('Adaline - Learning rate 0.01')

ada2 = AdalineSGD(n_iter=10, eta=0.0001).fit(X_std, y)
ax[1].plot(range(1, len(ada2.cost_) + 1), ada2.cost_, marker='o')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Sum-squared-error')
ax[1].set_title('Adaline - Learning rate 0.0001')

# plt.savefig('images/02_11.png', dpi=300)
plt.show()

We will set initial learning rate (eta) to 0.01

In [ ]:
ada_sgd = AdalineSGD(n_iter=15, eta=0.01)
ada_sgd.fit(X_std, y)

plot_decision_regions(X_std, y, classifier=ada_sgd)
plt.title('Adaline: Setosa vs Versicolor (2 features)')
plt.xlabel('sepal length [Standardized]')
plt.ylabel('petal length [Standardized]')
plt.legend(["setosa","versicolor"],loc="upper left")
plt.tight_layout()
plt.show()

plt.plot(range(1, len(ada_sgd.cost_)+1),ada_sgd.cost_,marker='o')
plt.xlabel('Epochs')
plt.ylabel('Sum-squared-error')
plt.tight_layout()
plt.show()

## Evaluating the model

In [ ]:
train_acc = ada_sgd.evaluate(X_std,y)
print("training set accuracy: %.2f%%" % (train_acc*100))

As shown on the graph, Adaline has converged after training with learning rate 0.01. However, Sum-squared-error (SSE) remains non zero even though all samples were classified correctly.

## 5. Repeat Step 4) using three features at a time.

Selected classes (2): 
- Iris-setosa
- Iris-versicolor

Selected features (3): 
- Sepal length
- Sepal Width
- Petal length

In [ ]:
X = df.loc[0:99, ["sepal_length","sepal_width","petal_length"]].values

In [ ]:
fig = plt.figure(figsize=(12,8))

ax = fig.add_subplot(projection='3d')

plt.scatter(X[:50,0], X[:50,1], X[:50,2], color="red", marker="o", label="setosa")
plt.scatter(X[50:100,0], X[50:100,1], X[50:100,2], color="blue", marker="x", label="versicolor")
ax.set_xlabel("sepal length (cm)")
ax.set_ylabel("sepal width (cm)")
ax.set_zlabel("petal length (cm)")

# set initial view
ax.view_init(50,50,50)
plt.title("setosa vs. versicolor (sepal length/ sepal width/ petal length)")
plt.legend(loc="upper left")
plt.show()

As shown in the plot, the data is linearly separable.

### Perceptron - linearly separable (3 features)

In [ ]:
ppn = Perceptron(eta=0.1, n_iter=10)
ppn.fit(X,y)

plt.plot(range(1, len(ppn.errors_)+1), ppn.errors_, marker="o")
plt.xlabel("epochs")
plt.ylabel("# of updates")
plt.title("Misclassification errors on each epochs")
plt.show()

As shown in the plot, perceptron converges after 5th epoch.

In [ ]:
# Plot a decision regions in 3D

from matplotlib.colors import ListedColormap
def plot_decision_regions_3D(X, y, classifier, resolution=0.02, v1=0, v2=0, v3=0):
    
    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # setup for 3D plot
    fig = plt.figure(figsize=(12,8))
    ax = plt.axes(projection="3d")
    ax.set_xlabel("sepal length (cm)")
    ax.set_ylabel("sepal width (cm)")
    ax.set_zlabel("petal length (cm)")
    
    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution), 
                           np.arange(x2_min, x2_max, resolution))

    # Get weights
    w = classifier.w_
    
    # projection of the plane
    Z = -(1/w[3])*(w[0]*xx1 + w[1]*xx2 + w[2])
    ax.plot_surface(xx1, xx2, Z, alpha=0.5, color="yellow")
    ax.view_init(v1,v2,v3)
    
    # plot class examples
    ax.scatter(X[:50,0], X[:50,1], X[:50,2], color="red", marker="o", label="setosa")
    ax.scatter(X[50:100,0], X[50:100,1], X[50:100,2], color="blue", marker="x", label="versicolor")


In [ ]:
plot_decision_regions_3D(X, y, classifier=ppn,v1=0,v2=10,v3=0)
plt.legend(["separation plane","setosa","versicolor"],loc="upper left")
plt.title("Perceptron: setosa vs. versicolor (3 features)")
plt.show()

## Evaluating the model

In [ ]:
train_acc = ppn.evaluate(X,y)
print("training set accuracy: %.2f%%" % (train_acc*100))

As we can see in the plot, the perceptron created a decision boundary that is able to classify all flower examples in the Iris training subset.

### Adaline - linearly separable (3 features)

In [ ]:
# Standardizing data
X_std = np.copy(X)
X_std[:,0] = (X[:,0] - X[:,0].mean()) / X[:,0].std()
X_std[:,1] = (X[:,1] - X[:,1].mean()) / X[:,1].std()
X_std[:,2] = (X[:,2] - X[:,2].mean()) / X[:,2].std()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

ada1 = AdalineSGD(n_iter=10, eta=0.1).fit(X_std, y)
ax[0].plot(range(1, len(ada1.cost_) + 1), np.log10(ada1.cost_), marker='o')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('log(Sum-squared-error)')
ax[0].set_title('Adaline - Learning rate 0.1')

ada2 = AdalineSGD(n_iter=10, eta=0.01).fit(X_std, y)
ax[1].plot(range(1, len(ada2.cost_) + 1), np.log10(ada2.cost_), marker='o')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('log(Sum-squared-error)')
ax[1].set_title('Adaline - Learning rate 0.01')

# plt.savefig('images/02_11.png', dpi=300)
plt.show()

Choosing learning rate (eta) of 0.1

In [ ]:
ada_sgd = AdalineSGD(n_iter=15, eta=0.1)
ada_sgd.fit(X_std, y)

plot_decision_regions_3D(X_std, y, classifier=ada_sgd, v1=0,v2=20,v3=0)
plt.title('Adaline: setosa vs. versicolor (3 features)')
plt.legend(["separation plane", "setosa","versicolor"],loc="upper left")
plt.tight_layout()
plt.show()

plt.plot(range(1, len(ada_sgd.cost_)+1),ada_sgd.cost_,marker='o')
plt.xlabel('Epochs')
plt.ylabel('Sum-squared-error')
plt.tight_layout()
plt.show()

## Evaluating the model

In [ ]:
train_acc = ada_sgd.evaluate(X_std,y)
print("training set accuracy: %.2f%%" % (train_acc*100))

As shown in the graph, Adaline is able to classify the data. 

## 6. Repeat Step 4) using all four features at the same time.

Selected classes (2): 
- Iris-setosa
- Iris-versicolor

Selected features (4): 
- Sepal length
- Sepal Width
- Petal length
- Petal Width

In [ ]:
XX = df.loc[0:99]
X = XX[["sepal_length","sepal_width","petal_length", "petal_width"]].values

In [ ]:
XX.shape

In [ ]:
# Scatter plot of the dataset
sns.pairplot(XX,hue="class_label")
plt.show()

As shown in the graphs, the data is linearly separable.

### Perceptron - linearly separable (4 features)

In [ ]:
ppn = Perceptron(eta=0.1, n_iter=10)
ppn.fit(X,y)

plt.plot(range(1, len(ppn.errors_)+1), ppn.errors_, marker="o")
plt.xlabel("epochs")
plt.ylabel("# of updates")
plt.title("Misclassification errors on each epochs")
plt.show()

As shown in the graph, perceptron converges after 4 epochs.

## Evaluating the model

In [ ]:
train_acc = ppn.evaluate(X,y)
print("training set accuracy: %.2f%%" % (train_acc*100))

As with the accuracy, Perceptron is able to classify the data sets. 

### Adaline - linearly separable (4 features)

In [ ]:
X_std = np.copy(X)
X_std[:,0] = (X[:,0] - X[:,0].mean()) / X[:,0].std()
X_std[:,1] = (X[:,1] - X[:,1].mean()) / X[:,1].std()
X_std[:,2] = (X[:,2] - X[:,2].mean()) / X[:,2].std()
X_std[:,3] = (X[:,3] - X[:,3].mean()) / X[:,3].std()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

ada1 = AdalineSGD(n_iter=10, eta=0.01).fit(X_std, y)
ax[0].plot(range(1, len(ada1.cost_) + 1), np.log10(ada1.cost_), marker='o')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('log(Sum-squared-error)')
ax[0].set_title('Adaline - Learning rate 0.01')

ada2 = AdalineSGD(n_iter=10, eta=0.0001).fit(X_std, y)
ax[1].plot(range(1, len(ada2.cost_) + 1), ada2.cost_, marker='o')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Sum-squared-error')
ax[1].set_title('Adaline - Learning rate 0.0001')

# plt.savefig('images/02_11.png', dpi=300)
plt.show()

In [ ]:
ada_sgd = AdalineSGD(n_iter=15, eta=0.01)
ada_sgd.fit(X_std, y)

plt.plot(range(1, len(ada_sgd.cost_)+1),ada_sgd.cost_,marker='o')
plt.xlabel('Epochs')
plt.ylabel('Sum-squared-error')
plt.tight_layout()
plt.show()

## Evaluating the model

In [ ]:
train_acc = ada_sgd.evaluate(X_std,y)
print("training set accuracy: %.2f%%" % (train_acc*100))

As with the number, Adaline has a 100% accuracy.

## 7. Pick two classes of data (i.e., two species of Iris) and two features from the four in the dataset, so that the data for two species are NOT linearly separable using the features that you have chosen.

Selected classes (2): 
- Iris-setosa
- Iris-verginica

Selected features (2): 
- Sepal length
- Petal length

Extract the last 100 class labels that correspond to the 50 Iris-versicolor and 50 Iris-setosa flowers

In [ ]:
y = df.loc[50:,"class_label"].values
y

In [ ]:
y = np.where(y=="Iris-versicolor",-1,1)
y

In [ ]:
X = df.loc[50:, ["sepal_length","petal_length"]].values

In [ ]:
plt.scatter(X[:50,0], X[:50,1], color="red", marker="o", label="versicolor")
plt.scatter(X[50:100,0], X[50:100,1], color="blue", marker="x", label="verginica")
plt.xlabel("sepal length (cm)")
plt.ylabel("petal length (cm)")
plt.title("versicolor vs. verginica (sepal length/ petal length)")
plt.legend(loc="upper left")
plt.show()

As shown in the graph, the selected data is not linearly separable.

### Perceptron - not linearly separable (2 features)

In [ ]:
ppn = Perceptron(eta=0.1, n_iter=10)
ppn.fit(X,y)
plt.plot(range(1, len(ppn.errors_)+1), ppn.errors_, marker="o")
plt.xlabel("epochs")
plt.ylabel("# of updates")
plt.title("Misclassification errors on each number of epochs")
plt.show()

As shown in the plot, perceptron is not converged.

In [ ]:
plot_decision_regions(X, y, classifier=ppn)
plt.xlabel("sepal length (cm)")
plt.ylabel("petal length (cm)")
plt.legend(["versicolor","verginica"],loc="upper left")
plt.title("Perceptron: versicolor vs. verginica (2 ")
plt.show()

## Evaluating the model

Can not calculate accuracy because the data is not linearly separable.

Since the data is not linearly separable, it is not possible to use Perceptron.

### Adaline - not linearly separable (2 features)

In [ ]:
X_std = np.copy(X)
X_std[:,0] = (X[:,0] - X[:,0].mean()) / X[:,0].std()
X_std[:,1] = (X[:,1] - X[:,1].mean()) / X[:,1].std()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

ada1 = AdalineSGD(n_iter=10, eta=0.01).fit(X_std, y)
ax[0].plot(range(1, len(ada1.cost_) + 1), np.log10(ada1.cost_), marker='o')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('log(Sum-squared-error)')
ax[0].set_title('Adaline - Learning rate 0.01')

ada2 = AdalineSGD(n_iter=10, eta=0.0001).fit(X_std, y)
ax[1].plot(range(1, len(ada2.cost_) + 1), ada2.cost_, marker='o')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Sum-squared-error')
ax[1].set_title('Adaline - Learning rate 0.0001')

# plt.savefig('images/02_11.png', dpi=300)
plt.show()

In [ ]:
ada_sgd = AdalineSGD(n_iter=15, eta=0.01)
ada_sgd.fit(X_std, y)

plot_decision_regions(X_std, y, classifier=ada_sgd)
plt.title('Adaline: versicolor vs. virginica (2 features)')
plt.xlabel('sepal length [Standardized]')
plt.ylabel('petal length [Standardized]')
plt.legend(["versicolor","virginica"],loc="upper left")
plt.tight_layout()
plt.show()

plt.plot(range(1, len(ada_sgd.cost_)+1),ada_sgd.cost_,marker='o')
plt.xlabel('Epochs')
plt.ylabel('Sum-squared-error')
plt.tight_layout()
plt.show()

## Evaluating the model

In [ ]:
train_acc = ada_sgd.evaluate(X_std,y)
print("training set accuracy: %.2f%%" % (train_acc*100))

As shown in the graph, Adaline method is better fitting the dataset.

## 8. Repeat Step 7) using three features at a time.

Selected classes (2): 
- Iris-versicolor
- Iris-virginica

Selected features (3): 
- Sepal length
- Sepal Width
- Petal length

In [ ]:
X = df.loc[50:, ["sepal_length","sepal_width","petal_length"]].values

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(projection='3d')

plt.scatter(X[:50,0], X[:50,1], X[:50,2], color="red", marker="o", label="versicolor")
plt.scatter(X[50:100,0], X[50:100,1], X[50:100,2], color="blue", marker="x", label="virginica")
ax.set_xlabel("sepal length (cm)")
ax.set_ylabel("sepal width (cm)")
ax.set_zlabel("petal length (cm)")

plt.title("versicolor vs. virginica (sepal length/ sepal width/ petal length)")
plt.legend(loc="upper left")
plt.show()

As shown in the graph, the data is not linearly separable.

### Perceptron - not linearly separable (3 features)

In [ ]:
ppn = Perceptron(eta=0.1, n_iter=10)
ppn.fit(X,y)

plt.plot(range(1, len(ppn.errors_)+1), ppn.errors_, marker="o")
plt.xlabel("epochs")
plt.ylabel("# of updates")
plt.title("Misclassification errors on each epochs")
plt.show()

As shown in the graph above, there is no convergence.

In [ ]:
plot_decision_regions_3D(X, y, classifier=ppn,v1=10,v2=0,v3=0)
plt.legend(["separation plane","versicolor","virginica"],loc="upper left")
plt.title("Perceptron: versicolor vs. virginica (3 features)")
plt.show()

## Evaluating the model

Can not calculate accuracy because the data is not linearly separable.

Since the data is not linearly separable, it is not possible to use Perceptron.

### Adaline - not linearly separable (3 features)

In [ ]:
X_std = np.copy(X)
X_std[:,0] = (X[:,0] - X[:,0].mean()) / X[:,0].std()
X_std[:,1] = (X[:,1] - X[:,1].mean()) / X[:,1].std()
X_std[:,2] = (X[:,2] - X[:,2].mean()) / X[:,2].std()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

ada1 = AdalineSGD(n_iter=10, eta=0.01).fit(X_std, y)
ax[0].plot(range(1, len(ada1.cost_) + 1), np.log10(ada1.cost_), marker='o')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('log(Sum-squared-error)')
ax[0].set_title('Adaline - Learning rate 0.01')

ada2 = AdalineSGD(n_iter=10, eta=0.0001).fit(X_std, y)
ax[1].plot(range(1, len(ada2.cost_) + 1), ada2.cost_, marker='o')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Sum-squared-error')
ax[1].set_title('Adaline - Learning rate 0.0001')

# plt.savefig('images/02_11.png', dpi=300)
plt.show()

In [ ]:
ada_sgd = AdalineSGD(n_iter=15, eta=0.01)
ada_sgd.fit(X_std, y)

plot_decision_regions_3D(X_std, y, classifier=ada_sgd, v1=0,v2=20,v3=0)
plt.title('Adaline: versicolor vs. virginica (3 features)')
plt.xlabel('sepal length')
plt.ylabel('petal length')
plt.legend(["separation plane","versicolor", "virginica"],loc="upper left")
plt.tight_layout()
plt.show()

plt.plot(range(1, len(ada_sgd.cost_)+1),ada_sgd.cost_,marker='o')
plt.xlabel('Epochs')
plt.ylabel('Sum-squared-error')
plt.tight_layout()
plt.show()

## Evaluating the model

In [ ]:
train_acc = ada_sgd.evaluate(X_std,y)
print("training set accuracy: %.2f%%" % (train_acc*100))

As shown in the plot, Adaline able to classify the data.

## 9. Repeat Step 7) using all four feature at the same time.

Selected classes (2): 
- Iris-versicolor
- Iris-virginica

Selected features (4): 
- Sepal length
- Sepal Width
- Petal length
- Petal Width


In [ ]:
XX = df.loc[50:]
X = XX[["sepal_length","sepal_width","petal_length", "petal_width"]].values

In [ ]:
# Scatter plot of the dataset
sns.pairplot(XX,hue="class_label")
plt.show()

As shown in the plot, all data are not linearly separable.

### Perceptron - not linearly separable (4 features)

In [ ]:
ppn = Perceptron(eta=0.1, n_iter=10)
ppn.fit(X,y)

plt.plot(range(1, len(ppn.errors_)+1), ppn.errors_, marker="o")
plt.xlabel("epochs")
plt.ylabel("# of updates")
plt.title("Misclassification errors on each epochs")
plt.show()

As shown in the graph, perceptron does not converged.

## Evaluating the model

Can not calculate accuracy because the data is not linearly separable.

Since the data is not linearly separable, it is not possible to use Perceptron.

### Adaline - not linearly separable (4 features)

In [ ]:
X_std = np.copy(X)
X_std[:,0] = (X[:,0] - X[:,0].mean()) / X[:,0].std()
X_std[:,1] = (X[:,1] - X[:,1].mean()) / X[:,1].std()
X_std[:,2] = (X[:,2] - X[:,2].mean()) / X[:,2].std()
X_std[:,3] = (X[:,3] - X[:,3].mean()) / X[:,3].std()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

ada1 = AdalineSGD(n_iter=10, eta=0.01).fit(X_std, y)
ax[0].plot(range(1, len(ada1.cost_) + 1), np.log10(ada1.cost_), marker='o')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('log(Sum-squared-error)')
ax[0].set_title('Adaline - Learning rate 0.01')

ada2 = AdalineSGD(n_iter=10, eta=0.0001).fit(X_std, y)
ax[1].plot(range(1, len(ada2.cost_) + 1), ada2.cost_, marker='o')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Sum-squared-error')
ax[1].set_title('Adaline - Learning rate 0.0001')

# plt.savefig('images/02_11.png', dpi=300)
plt.show()

In [ ]:
ada_sgd = AdalineSGD(n_iter=15, eta=0.01)
ada_sgd.fit(X_std, y)

plt.plot(range(1, len(ada_sgd.cost_)+1),ada_sgd.cost_,marker='o')
plt.xlabel('Epochs')
plt.ylabel('Sum-squared-error')
plt.tight_layout()
plt.show()

## Evaluating the model

In [ ]:
train_acc = ada_sgd.evaluate(X_std,y)
print("training set accuracy: %.2f%%" % (train_acc*100))

As shown in the graph, unlike Perceptron, Adaline is better suit for Non linearly Separable data.

## Summary Tables

- ### Perceptron

|  Species 1 | Species 2  | Sepal length | Sepal width | Petal length | Petal width | Result            | Accuracy  |
|------------|------------|--------------|-------------|--------------|-------------|-------------------|-----------|
| setosa     | versicolor | o            |             | o            |             | converged         |   100%    |
| setosa     | versicolor | o            | o           | o            |             | converged         |   100%    |
| setosa     | versicolor | o            | o           | o            |  o          | converged         |   100%    |
| versicolor | virginica  | o            |             | o            |             | not converged     |   N/A     |
| versicolor | virginica  | o            | o           | o            |             | not converged     |   N/A     |
| versicolor | virginica  | o            | o           | o            |  o          | not converged     |   N/A     |

- ### Adaline

|  Species 1 | Species 2  | Sepal length | Sepal width | Petal length | Petal width | Result            | Accuracy  |
|------------|------------|--------------|-------------|--------------|-------------|-------------------|-----------|
| setosa     | versicolor | o            |             | o            |             | converged         |   100%    |
| setosa     | versicolor | o            | o           | o            |             | converged         |   100%    |
| setosa     | versicolor | o            | o           | o            |  o          | converged         |   100%    |
| versicolor | virginica  | o            |             | o            |             | converged         |    95%    |
| versicolor | virginica  | o            | o           | o            |             | converged         |    94%    |
| versicolor | virginica  | o            | o           | o            |  o          | converged         |    97%    |


## Conclusion

- ### Perceptron
    - Binary Linear Classifier
    - Use step function
    - Only work with linearly separable data sets.
    - Does not converge if classes are not linearly separable
    - Guaranteed to converge to a solution if the data is linearly separable. However, it might not be the most optimal one.


  - ### Adaline
    - Also a Binary Linear Classifier
    - Use Gradient Descent
    - Can work with non linearly separable data sets.
    - Always converges regardless of linearly separable or not
    - Always try to optimizes the solution with minimal errors